In [6]:
import CommitASTModule as cm
import requests
from your_token import token

user = "BroadleafCommerce"
repo = "BroadleafCommerce"
headers = {"Accept": "application/vnd.github+json", "Authorization" : f"Token {token}"}
baseURL = f"https://api.github.com/repos/{user}/{repo}"

### 1. Get the list of commits for each pull request implementing a feature

In [7]:
pull_requests = requests.get(f"{baseURL}/pulls", params={"labels" : ["type-feature"], "state" : "closed"}, headers=headers).json()
commits_per_pr = { p["number"] : requests.get(p["commits_url"], headers=headers).json() for p in pull_requests}


for key, val in commits_per_pr.items():
    commits_per_pr[key] = [requests.get(c["url"], headers=headers).json() for c in val]

### 2. Get each file that has bean modified by 2 pull requests

#### 2.1 Get the list of files modified by each pull request

*file_per_pr* = {pr_id: [(file1, commit), (file2, commit) ...], ...}

In [8]:
file_per_pr = {}
for key, val in commits_per_pr.items():
    files = []
    for c in commits_per_pr[key]:
        files += [(f["filename"], c["sha"]) for f in c["files"]]
    file_per_pr[key] = files

#### 2.2 Search for intersections between the lists of files in *file_per_pr*

In [9]:
def find_intersection(list1, list2):
    result = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if list1[i][0] == list2[j][0]:
                result.append((list1[i], list2[j]))
    return result


intersections = []
keys = list(commits_per_pr.keys())
for i in range(len(keys)):
    for j in range(i+1, len(keys)):
        inter = find_intersection(file_per_pr[keys[i]], file_per_pr[keys[j]])
        if len(inter) > 0:
            intersections.append((keys[i], keys[j], inter)) 

cm.printJson(intersections[5])

[
    2823,
    2816,
    [
        [
            [
                "admin/broadleaf-open-admin-platform/src/main/java/org/broadleafcommerce/openadmin/server/service/persistence/validation/AdminRegexValidator.java",
                "0e5aca2181318414618d689bde3d6eb5ba863185"
            ],
            [
                "admin/broadleaf-open-admin-platform/src/main/java/org/broadleafcommerce/openadmin/server/service/persistence/validation/AdminRegexValidator.java",
                "af9db6a4504a3c9eb59e7d898d4f128a9f7a75de"
            ]
        ],
        [
            [
                "admin/broadleaf-open-admin-platform/src/main/java/org/broadleafcommerce/openadmin/server/service/persistence/validation/AdminRegexValidator.java",
                "0e5aca2181318414618d689bde3d6eb5ba863185"
            ],
            [
                "admin/broadleaf-open-admin-platform/src/main/java/org/broadleafcommerce/openadmin/server/service/persistence/validation/AdminRegexValidator.java",
        

### 3. Get 2 commits intersecting in a file

In [10]:
inter_example = intersections[5]

filename = inter_example[2][0][0][0]
ref1 = inter_example[2][0][0][1]
ref2 = inter_example[2][0][1][1]
commit1 = requests.get(f"{baseURL}/commits/{ref1}", headers=headers).json()
commit2 = requests.get(f"{baseURL}/commits/{ref2}", headers=headers).json()

file1 = None
file2 = None

for file in commit1["files"]:
    if file["filename"] == filename:
        file1 = file

for file in commit2["files"]:
    if file["filename"] == filename:
        file2 = file

#### 4. The next steps would be to see what part of the code belong to each commits
The aim is to determine the relation between two features analyeing the asts 